In [1]:
import pandas as pd
import numpy as np
import random

In [2]:
TAZ = pd.read_csv('TAZ Data - Sheet1.csv').to_numpy()
TAZ_num = TAZ.shape[0]
zone_name = pd.read_csv('TAZ Data - Sheet2.csv',header=None).to_numpy()
print(TAZ)
print(zone_name)

[[  1.           6.           9.           2.88888889   4.66666667]
 [  2.           2.           3.           2.           4.        ]
 [  3.           3.           1.           2.           7.        ]
 [  4.           1.         247.           8.49797571  18.15789474]
 [  5.           6.           7.           2.28571429  21.42857143]
 [  6.           5.           2.           2.          24.5       ]
 [  7.           9.           3.           2.33333333  26.66666667]
 [  8.           8.           4.           2.25        28.75      ]
 [  9.           9.           3.           2.          32.        ]
 [ 10.           3.           1.           3.           8.        ]
 [ 11.           4.          18.           5.5         30.        ]
 [ 12.           2.           3.           4.          32.        ]
 [ 13.           6.           9.           7.          33.        ]
 [ 14.           5.           3.           4.           3.        ]
 [ 15.           9.           4.           4.5  

In [3]:
# demographics
tp = 500000 #total population
under_5 = 4.5/100
under_18 = 13.4/100
college_student = 20000/tp
over_65 = 15.1/100

k12 = under_18-under_5
working_age = 1 - (under_18+college_student+over_65)
demographics = [under_5, k12, college_student, working_age, over_65]

unemployment = 0.082
employed_pop = (1-unemployment)*demographics[3]*tp
unemployed_pop = unemployment*demographics[3]*tp

In [4]:
pixel_marketcap = []
for i in range(1,13):
    pixel_marketcap.append(sum(TAZ[TAZ[:,1]==i][:,2]))
print(pixel_marketcap)

[498.0, 60.0, 30.0, 18.0, 39.0, 188.0, 21.0, 22.0, 80.0, 38.0, 18.0, 12.0]


In [5]:
class AP:
  def __init__(self, pop, P_zones =None, A_zones =None, A_dist =np.ones(12)/12):
    self.P_zones = np.array(P_zones)
    self.A_zones = np.array(A_zones)
    self.pop = pop
    self.A_dist = A_dist

  def equalize(self):
    """makes the size of the attraction vector equal to the size of the production vector"""
    if sum(self.P) != sum(self.A):
        disparity = sum(self.P) - sum(self.A)
#         print(sum(P), sum(A),disparity)
        k = 0
        while (k == 0):
            ind_draw = random.randint(0,TAZ_num-1)
            k = self.A[ind_draw]
        self.A[ind_draw] = self.A[ind_draw] + disparity
    return
    
  def apv(self):
    """creates attraction and production vectors"""
    P_total_pixel_mrktcap = sum(np.take(pixel_marketcap, self.P_zones-1))
    A_total_pixel_mrktcap = sum(np.take(pixel_marketcap, self.A_zones-1))
#         print(P_total_pixel_mrktcap, A_total_pixel_mrktcap)
    #indicator      
    P = np.array([a in self.P_zones for a in TAZ[:,1]]).astype(int)/P_total_pixel_mrktcap
    A = np.array([a in self.A_zones for a in TAZ[:,1]]).astype(int)/A_total_pixel_mrktcap
#         print(P*TAZ[:,2])
#         print(A*TAZ[:,2])
    self.P = np.around(self.pop*TAZ[:,2]*P)
    self.A = np.around(self.pop*TAZ[:,2]*A)
    return
    
  def pv(self):
    """creates production vector from zone denotes"""
    P_total_pixel_mrktcap = sum(np.take(pixel_marketcap, self.P_zones-1))
    P = np.array([a in self.P_zones for a in TAZ[:,1]]).astype(int)/P_total_pixel_mrktcap
    self.P = np.around(self.pop*TAZ[:,2]*P)
    return
    
  def av(self):
    """creates attraction vector from zone denotes"""
    A_total_pixel_mrktcap = sum(np.take(pixel_marketcap, self.A_zones-1))
    A = np.array([a in self.A_zones for a in TAZ[:,1]]).astype(int)/A_total_pixel_mrktcap
    self.A = np.around(self.pop*TAZ[:,2]*A)
    return

  def a_dist(self):
    """creates attraction vector from distribution of demands"""
    A = np.take(self.A_dist, (np.array(TAZ[:,1])-1).astype(int))/np.take(pixel_marketcap, (np.array(TAZ[:,1])-1).astype(int))
    A = np.around(self.pop*TAZ[:,2]*A)
    self.A = A
    return

  def df(self):
        """returns pairwise dataframe"""
        return np.column_stack((self.P,self.A))
        

#### Object Tests

In [6]:
x = AP(employed_pop,[6], [10,11])
x.apv()
print(sum(x.P))
print(sum(x.A))
x.equalize()
print(sum(x.P))
print(sum(x.A))

309824.0
309826.0
309824.0
309824.0


In [7]:
x = AP(pop =(unemployed_pop+demographics[4]*tp), A_dist =[0,0.07,0.10,0,0,0,0.13,0.06,0.56,0,0.06,0.02])
x.a_dist()
print(x.A)

[    0.   361.   344.     0.     0.     0.  2167.  1126.  2167.   344.
     0.   361.     0.     0.  2889.     0.     0.  2167.   344.     0.
   688.     0.     0.   602.     0.  1444.   563.     0.   344.     0.
   120.     0.     0.  1444.  3611.   563.   281.     0.  4333.  1032.
     0.   516.     0.   516.  1407.     0.     0.  2407.  1032.     0.
 16611.  2064.   844.     0.     0.  4471.     0.     0.  1376.   602.
   722.     0.  2528.     0. 13413. 12278.     0.     0.  3095.   563.
     0.     0.  7944.   844.  1926.     0.   722.     0.]


### Row 1

In [8]:
home_to_work = AP(employed_pop, [6],[10,11])
home_to_work.apv()
home_to_work.equalize()

### Row 2

In [9]:
home_to_k12 = AP((demographics[1])*tp,[6],[5])
home_to_k12.apv()
home_to_k12.equalize()
home_to_college = AP((demographics[2])*tp,[6],[4])
home_to_college.apv()
home_to_college.equalize()

home_to_school = AP(demographics[1]+demographics[2])
home_to_school.P = home_to_k12.P + home_to_college.P
home_to_school.A = home_to_k12.A + home_to_college.A

### Row 3

In [10]:
recreational_demand_from_home = [0,0.07,0.10,0,0,0,0.13,0.06,0.56,0,0.06,0.02]
home_to_SR_unemployed = AP(pop =(unemployed_pop+demographics[4]*tp), P_zones =[6], A_dist =recreational_demand_from_home)
home_to_SR_unemployed.pv()
home_to_SR_unemployed.a_dist()
home_to_SR_unemployed.equalize()

home_to_SR_employed = AP(pop =0.3*(demographics[1]+demographics[2])*tp +0.5*employed_pop, P_zones =[6], A_dist =recreational_demand_from_home)
home_to_SR_employed.pv()
home_to_SR_employed.a_dist()
home_to_SR_employed.equalize()

### Row 4

In [11]:
work_to_home = AP(pop =sum(np.around(0.6*home_to_work.A)), A_zones =[6])
work_to_home.P = np.around(0.6*home_to_work.A)
work_to_home.av()
work_to_home.equalize()

### Row 5

In [12]:
school_to_home = AP(pop =sum(np.around(0.75*home_to_school.A)), A_zones =[6])
school_to_home.P = np.around(0.75*home_to_school.A)
school_to_home.av()
school_to_home.equalize()

### Row 6

In [13]:
SR_to_home_unemployed = AP(pop =sum(np.around(0.50*home_to_SR_unemployed.A)), A_zones =[6])
SR_to_home_unemployed.P = np.around(0.50*home_to_SR_unemployed.A)
SR_to_home_unemployed.av()
SR_to_home_unemployed.equalize()

SR_to_home_employed = AP(pop =sum(np.around(0.50*home_to_SR_employed.A)), A_zones =[6])
SR_to_home_employed.P = np.around(0.50*home_to_SR_employed.A)
SR_to_home_employed.av()
SR_to_home_employed.equalize()

### Row 7

In [14]:
work_to_lunch = AP(pop =sum(np.around(0.10*home_to_work.A)), A_zones =[9])
work_to_lunch.P = np.around(0.10*home_to_work.A)
work_to_lunch.av()
work_to_lunch.equalize()

lunch_to_work = AP(sum(np.around(0.10*home_to_work.A)))
lunch_to_work.P = work_to_lunch.A
lunch_to_work.A = work_to_lunch.P

work_to_SR = AP(pop =sum(np.around(0.40*home_to_work.A)), A_dist =[0,0.10,0.13,0,0,0,0.04,0.15,0.56,0,0.00,0.02])
work_to_SR.P = np.around(0.40*home_to_work.A)
work_to_SR.a_dist()
work_to_SR.equalize()

work_SR_to_home = AP(pop =sum(np.around(0.40*home_to_work.A)), A_zones=[6])
work_SR_to_home.P = work_to_SR.A
work_SR_to_home.av()
work_SR_to_home.equalize()

### Row 8

In [15]:
school_to_SR = AP(pop =sum(np.around(0.25*home_to_school.A)), A_dist =[0,0.04,0.13,0,0,0,0.23,0.01,0.52,0,0.02,0.05])
school_to_SR.P = np.around(0.25*home_to_school.A)
school_to_SR.a_dist()
school_to_SR.equalize

school_SR_to_home = AP(pop =sum(np.around(0.25*home_to_school.A)), A_zones=[6])
school_SR_to_home.P = school_to_SR.A
school_SR_to_home.av()
school_SR_to_home.equalize()

### Row 9 (more shopping for shoppers)

In [16]:
SR_to_SR_unemployed = AP(pop =sum(home_to_SR_unemployed.A), A_dist =[0,0.10,0.15,0,0,0,0.15,0.10,0.42,0,0.07,0.01])
SR_to_SR_unemployed.P = home_to_SR_unemployed.A
SR_to_SR_unemployed.a_dist()
SR_to_SR_unemployed.equalize()

SR_to_SR_employed = AP(pop=sum(home_to_SR_employed.A), A_dist=[0,0.10,0.15,0,0,0,0.15,0.10,0.42,0,0.07,0.01])
SR_to_SR_employed.P = home_to_SR_employed.A
SR_to_SR_employed.a_dist()
SR_to_SR_employed.equalize()

SR_SR_to_home_unemployed = AP(pop =sum(home_to_SR_unemployed.A), A_zones=[6])
SR_SR_to_home_unemployed.P = SR_to_SR_unemployed.A
SR_SR_to_home_unemployed.av()
SR_SR_to_home_unemployed.equalize

SR_SR_to_home_employed = AP(pop =sum(home_to_SR_employed.A), A_zones=[6])
SR_SR_to_home_employed.P = SR_to_SR_employed.A
SR_SR_to_home_employed.av()
SR_SR_to_home_employed.equalize()

In [17]:
vectors =  [home_to_work,
            home_to_k12,
            home_to_college,
            home_to_school,
            home_to_SR_unemployed,
            home_to_SR_employed,
            work_to_home,
            school_to_home,
            SR_to_home_unemployed,
            SR_to_home_employed,
            work_to_lunch,
            lunch_to_work,
            work_to_SR,
            work_SR_to_home,
            school_to_SR,
            school_SR_to_home,
            SR_to_SR_unemployed,
            SR_to_SR_employed,
            SR_SR_to_home_unemployed,
            SR_SR_to_home_employed]
strings =  ['home_to_work',
            'home_to_k12',
            'home_to_college',
            'home_to_school',
            'home_to_SR_unemployed',
            'home_to_SR_employed',
            'work_to_home',
            'school_to_home',
            'SR_to_home_unemployed',
            'SR_to_home_employed',
            'work_to_lunch',
            'lunch_to_work',
            'work_to_SR',
            'work_SR_to_home',
            'school_to_SR',
            'school_SR_to_home',
            'SR_to_SR_unemployed',
            'SR_to_SR_employed',
            'SR_SR_to_home_unemployed',
            'SR_SR_to_home_employed']

In [18]:
for v in vectors:
    print((v.df()<0).any())

False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False


In [19]:
i=0
for v in vectors:
    pd.DataFrame(v.df()).to_csv(strings[i]+".csv",header=["P_"+strings[i],"A_"+strings[i]])
    i+=1

In [20]:
pd.DataFrame(vectors[1].df())

,0,1
0,2130.0,0.0
1,0.0,0.0
2,0.0,0.0
3,0.0,0.0
4,1657.0,0.0
...,...,...
73,0.0,0.0
74,0.0,0.0
75,0.0,4564.0
76,0.0,0.0
